The idea here was to investigate the exact distances of different ultras around the world. It's inetersting to see how they compare between Europe and US, for example. Are they exactly 100miles?

In [ ]:
%%capture
!pip install gpxpy haversine gdown

In [ ]:
import json
import gdown
import requests
import gpxpy
from haversine import haversine

In [21]:
urls = {
    "Western States 100": "https://www.wser.org/wp-content/uploads/gpx/WSER2025welev.gpx",
    "Hardrock 100": "https://hardrock100.com/files/course/HR100-Course-Counter-Clockwise.gpx",
    "Comrades Marathon": "https://www.goandrace.com/gpx/2025/06/08/gpx_20250608_id10326_race1_20250531165031.gpx?t=1765515595",
    "UTMB": "https://res.cloudinary.com/utmb-world/raw/upload/v1724242142/montblanc/Races/GPX/2024/UTMB_2024_V2108_49b6ad7dcc.gpx",
    "Mallorca by UTMB": "https://res.cloudinary.com/utmb-world/raw/upload/v1760605599/mallorca/GPX%20TRACKS/TRACKS%20DEFINITIVOS%20WEB%20MONICA%2015.10.25/100_M_SDT_2025_9207594015.gpx",
    "HOKA Val d'Aran by UTMB": "https://res.cloudinary.com/utmb-world/raw/upload/v1718621782/valdaran/Races/GPX%202024/VDA_2024_5ab5a38e62.gpx",
}

In [22]:
def download_gpx(url, filename):
    if url.startswith("https://drive.google.com/"):
        gdown.download(url, filename, quiet=False)
    else:
      response = requests.get(url, stream=True)
      if response.status_code != 200:
        return

      with open(filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

def process_gpx(filename):
    with open(filename, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        total_distance = 0
        for track in gpx.tracks:
            for segment in track.segments:
                for i in range(len(segment.points) - 1):
                    total_distance += haversine((segment.points[i].latitude, segment.points[i].longitude),
                                                (segment.points[i+1].latitude, segment.points[i+1].longitude))
        return total_distance

for filename, url in urls.items():
    download_gpx(url, filename)


total_distance = 0
for filename in urls.keys():
    distance = process_gpx(filename)
    total_distance += distance
    print(f"{filename}: {distance} km")


Western States 100: 161.05138538151638 km
Hardrock 100: 159.24123858452833 km
Comrades Marathon: 90.91605911785616 km
UTMB: 173.92642439259075 km
Mallorca by UTMB: 137.45744441704628 km
HOKA Val d'Aran by UTMB: 159.8179572250338 km
